In [ ]:
import random
import torch
import math
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import gym as gym
from gym import spaces
import carla
import numpy as np
from collections import deque
from stable_baselines3 import SAC
from stable_baselines3.sac.policies import MlpPolicy
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env
import csv
from time import sleep

In [ ]:
threshold = 2
CSV_FILE = '/home/kololhe/stuff/waypoints.csv'


In [ ]:
def read_waypoints(file_path):
    waypoints = []
    with open(file_path, 'r') as csvfile:
        csvreader = csv.reader(csvfile)
        for row in csvreader:
            x, y, yaw = map(float, row)
            waypoints.append((x, y, yaw))
            
    return waypoints


def setupCarla():

    # Connect to CARLA server
    client = carla.Client('localhost', 2000)
    client.set_timeout(10.0)

    return client.get_world()


In [ ]:
class CarlaInstance(gym.Env):
    metadata = {}

    def __init__(self, world, waypoints):
        #super(CarlaInstance, self).__init__()
        try:
            self.waypoints = waypoints
            self.current_waypoint_index = 0

            self.blueprint_library = world.get_blueprint_library()
            world.set_weather(carla.WeatherParameters.ClearNoon)


            # Spawn vehicle
            vehicle_bp = random.choice(self.blueprint_library.filter('wrangler_rubicon'))
            spawn_point = carla.Transform(carla.Location(x=-23.6,y=137.5,z=1),carla.Rotation(yaw=0))
            self.vehicle = world.spawn_actor(vehicle_bp, spawn_point)
            #self.vehicle.set_simulate_physics(True)
            self.vehicle.apply_control(carla.VehicleControl(throttle=1.0, brake=0.0, steer=0.0))
            sleep(2.0)

            # Attach Lane Invasion Sensor to car
            sensor_bp = world.get_blueprint_library().find('sensor.other.lane_invasion')
            sensor_transform = carla.Transform(carla.Location(x=2.5, z=0.7))
            self.lane_invasion_sensor = world.spawn_actor(sensor_bp, sensor_transform, attach_to=self.vehicle)
            self.lane_invasion_sensor.listen(lambda event: self.on_lane_invasion(event))

            self.collision_sensor = world.spawn_actor(world.get_blueprint_library().find('sensor.other.collision'), sensor_transform, attach_to=self.vehicle)
            self.collision_sensor.listen(lambda event: self.on_lane_invasion(event))

            self.action_space = spaces.Box(low = 0.0, high=1.0, shape=(1,), dtype=np.float32)
            self.observation_space = spaces.Box(low=-500000, high=500000, shape=(5,), dtype=np.float64)
            self.problem = bool
            #self.reset()

        except RuntimeError or KeyboardInterrupt:
            print("entered init exception")
            for actor in world.get_actors():
                actor.destroy()
            pass

    def on_lane_invasion(self, event):
        self.problem = True 
        #print("Theek kar")
        self.reset()
    
    def step(self, action):

        #print("{}\n", format(action))
        
        #throttle = action[0]
        #steer = action[1]
        #brake = 0
        # print(action)
        # if action.any() == None:
        #     reward = -10000
        
        throttle = action[0].astype(np.float32)
        steer = 0
        brake = 0

        control = carla.VehicleControl(throttle=throttle, brake=brake, steer=steer) #, steer=steer)
        self.vehicle.apply_control(control)
        
        #Getting next waypoint
        next_state = self.get_state()
        
        # Calculate reward
        reward = self.calculate_reward(next_state)

        # Check if the episode is done
        done = self.is_done(next_state)

        return next_state, reward, done
        
        #next_waypoint = self.waypoints[self.current_waypoint_index]
        #next_waypoint_location = carla.Location(x=next_waypoint[0], y=next_waypoint[1])
#         current_location = self.vehicle.get_location()
#         distance = current_location.distance(next_waypoint_location)

#         if distance < threshold:
#             self.current_waypoint_index += 1
#             if self.current_waypoint_index >= len(self.waypoints):
#                 self.current_waypoint_index = 0

#         reward = self.get_reward(distance, control)
#         done = False
#         if self.current_waypoint_index == len(self.waypoints) - 1 or self.problem:
#             done = True
# #         print (f'step taken: Throttle: {throttle} and steer: {steer}')
#         info = {}
#         sleep(0.05)

        #return self.get_observation(), reward, done, info
    
    def get_state(self):
        state = []
        curr_yaw = self.vehicle.get_transform().rotation.yaw


        for i in range(10):
            if self.current_waypoint + i < len(self.waypoints):
                waypoint = self.waypoints[self.current_waypoint + i]
                curr_vel = self.vehicle.get_velocity()
                curr_loc = self.vehicle.get_location()
                vx = curr_vel.x
                vy = curr_vel.y
                v = ((vx)**2+(vy)**2)**0.5
                distance = curr_loc.distance(carla.Location(x=waypoint[4],y=waypoint[5]))
                state.extend([v, curr_yaw - waypoint[1], curr_yaw - waypoint[2], distance ])
            else:
                state.extend([0, 0, 0, 0])
        return np.array(state)
    
    
#     def get_observation(self):
#         location = self.vehicle.get_location()
#         orientation = self.vehicle.get_transform().rotation.yaw
#         speed = self.vehicle.get_velocity()
#         speed = np.sqrt(speed.x**2 + speed.y**2 + speed.z**2)
#         next_waypoint = self.waypoints[self.current_waypoint_index]
#         next_waypoint_location = carla.Location(x=next_waypoint[0], y=next_waypoint[1])
#         distance = location.distance(next_waypoint_location)
# #         print('observation taken')

#         return np.array([location.x, location.y, orientation, speed, distance], dtype=np.float64)
    
#     def get_reward(self, distance, control):
#         reward = 0
#         if control.throttle > 0.9:
#             reward += 5
#         if distance < threshold:        
#             reward += distance+100    
#         if distance == 0:
#             reward += 1000
#         if self.vehicle.get_location() == carla.Transform(carla.Location(x=-23.6,y=137.5,z=1),carla.Rotation(yaw=0)):
#             reward -=500

#         # Apply a large negative reward for lane invasion
# #         print('reward calculated')    
#         return reward
    def calculate_reward(self, state):
        reward = -(np.linalg.norm(state[0] - self.waypoints[self.current_waypoint][0]) - (np.linalg.norm (state[1])) - (np.linalg.norm ( state[2])) - (np.linalg.norm (state[3])) + self.current_waypoint)
        return reward

    def is_done(self, state):
        if self.e == 1:
            self.current_waypoint += 1
            self.e=2
        if np.abs(state[3]) < 0.20:  # If the vehicle is close to the next waypoint
            self.current_waypoint += 1

        if self.current_waypoint >= len(self.waypoints):
            return True

        return False
    
    def close(self):
        self.vehicle.destroy()
        self.lane_invasion_sensor.destroy()
        self.collision_sensor.destroy()    

    def reset(self, seed=None, options=None):
        vel = carla.Vector3D()
        vel.x = 0
        vel.y = 0
        vel.z = 0
        self.vehicle.set_target_velocity(vel)
        x, y, yaw = self.waypoints[0]
        transform = carla.Transform(carla.Location(x=-23.6,y=137.5),carla.Rotation(yaw=0))
        self.vehicle.set_transform(transform)
        self.current_waypoint_index = 0        

        info = {}

        return self.get_observation()
    
    def render(self):
        pass


In [ ]:
# Create a vectorized environment with SubprocVecEnv
num_envs=1
env = CarlaInstance(world = setupCarla(), waypoints = read_waypoints(CSV_FILE))

check_env(env)

# Instantiate the PPO agent
model = SAC("MlpPolicy", env, verbose=1, gamma=0.99, learning_rate=0.0005, target_update_interval=1000, tau= 0.005)

# Train the agent
model.learn(total_timesteps=1000000, progress_bar=True, reset_num_timesteps=50000)

# Save the trained model
model.save("carla_ppo")